In [95]:
import numpy as np

p = np.random.randint( 3, 10 )
q = np.random.randint( 3, 10 )
r = min( p, q )

# construct a random matrix of size m, n
A = np.random.randn( p, q ) + 1j*np.random.randn( p, q )

# retrieve a first SVD representation
U, s, Vt = np.linalg.svd( A, full_matrices=False )

S = np.zeros(( r, r ), dtype=complex)
S[:r, :r] = np.diag(s)

In [96]:
# select randomly a column from the initial matrix (to be removed)
k = np.random.randint(q)
a = -A[ :, k ].reshape( p,1 )

# new matrix will have one extra column entry (update)
b = np.zeros( (q,1) ); b[ k ] = 1

# zero the column corresponding to the entry being removed
Vt[ :, k ] = 0

In [97]:
# construct the K matrix and its SVD representation
nv = np.dot( Vt, b )
Q = (b - np.dot( np.transpose(Vt), nv )) / np.sqrt( 1-np.dot( np.transpose(nv), nv ) )

# construct a new S matrix (padding with a row and column of zeros)
Sx = np.zeros( (r+1, r+1), dtype=complex )
Sx[ 0:r, 0:r ] = S


K = np.dot(
    
    Sx,
    (
        +np.identity( r+1, dtype=complex)
        -np.dot(
            np.append( np.dot( S, nv ), np.zeros( (1,1), dtype=complex ), axis= 0 ),
            np.transpose( np.append( nv, np.sqrt( 1-np.dot( np.transpose(nv), nv ) ), axis=0 ) )
        )
    )
)

Up, sp, Vpt = np.linalg.svd( K, full_matrices=False )

Sp = np.zeros(( r+1, r+1 ), dtype=complex)
Sp[:r+1, :r+1] = np.diag(sp)

In [100]:
# verify that the reconstructed matrix (using the SVD decomposition) matches the operation
Aab = A+np.dot( a, np.transpose(b))

Ai = np.dot(
    np.dot( np.append( U, np.zeros( (p,1) ), axis=1 ), Up ), Sp 
)

Ai = np.dot( 
    Ai, np.dot( 
        Vpt, np.transpose( np.append( np.transpose(Vt), Q, axis=1 ) )
    ) 
)

np.allclose( Aab, Ai )